# TensorFlow
### digits recognization

In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [5]:
mnist = input_data.read_data_sets('/tmp/data', one_hot=True)

Extracting /tmp/data\train-images-idx3-ubyte.gz
Extracting /tmp/data\train-labels-idx1-ubyte.gz
Extracting /tmp/data\t10k-images-idx3-ubyte.gz
Extracting /tmp/data\t10k-labels-idx1-ubyte.gz


In [6]:
n_nodes_hl1 = 500
n_nodes_hl2 = 500
n_nodes_hl3 = 500
n_classes = 10
batch_size = 100

In [14]:
x = tf.placeholder('float', [None, 784])
y = tf.placeholder('float')

In [47]:
def neural_network_model(data):
    hidden_1_layer = {'weights': tf.Variable(tf.random_normal([784, n_nodes_hl1])),
                      'biases': tf.Variable(tf.random_normal([n_nodes_hl1]))}
    hidden_2_layer = {'weights': tf.Variable(tf.random_normal([n_nodes_hl1, n_nodes_hl2])),
                      'biases': tf.Variable(tf.random_normal([n_nodes_hl2]))}
    hidden_3_layer = {'weights': tf.Variable(tf.random_normal([n_nodes_hl2, n_nodes_hl3])),
                      'biases': tf.Variable(tf.random_normal([n_nodes_hl2]))}
    output_layer = {'weights': tf.Variable(tf.random_normal([n_nodes_hl3, n_classes])),
                      'biases': tf.Variable(tf.random_normal([n_classes]))}
    
    l1 = tf.add(tf.matmul(data, hidden_1_layer['weights']), hidden_1_layer['biases'])
    l1 = tf.nn.relu(l1)# rectified linear
    l2 = tf.add(tf.matmul(l1, hidden_2_layer['weights']), hidden_2_layer['biases'])
    l2 = tf.nn.relu(l2)
    l3 = tf.add(tf.matmul(l2, hidden_3_layer['weights']), hidden_3_layer['biases'])
    l3 = tf.nn.relu(l3)
    output = tf.add(tf.matmul(l3, output_layer['weights']), output_layer['biases'])
    return output

In [54]:
def train_neural_network(x):
    prediction = neural_network_model(x)
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = prediction, labels = y))
    optimizer = tf.train.AdamOptimizer().minimize(cost)
    
    hm_epochs = 10
    
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
# initialize_all_variables (from tensorflow.python.ops.variables) is deprecated and will be removed after 2017-03-02.
# Instructions for updating:
# Use `tf.global_variables_initializer` instead.
        for epoch in range(hm_epochs):
            epoch_loss = 0
            for _ in range(int(mnist.train.num_examples/batch_size)):
                epoch_x, epoch_y = mnist.train.next_batch(batch_size)
                _, c = sess.run([optimizer, cost], feed_dict = {x:epoch_x, y:epoch_y})
                epoch_loss += c
            print('Epoch',epoch,'completed out of',hm_epochs,'loss',epoch_loss)
        
        correct = tf.equal(tf.argmax(prediction,1),tf.argmax(y,1))
        accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
        print('Accuracy', accuracy.eval({x:mnist.test.images, y:mnist.test.labels}))
    

In [55]:
train_neural_network(x)

Epoch 0 completed out of 10 loss 1802291.43850708
Epoch 1 completed out of 10 loss 411481.5880432129
Epoch 2 completed out of 10 loss 222302.706782341
Epoch 3 completed out of 10 loss 132496.39847421646
Epoch 4 completed out of 10 loss 77331.07215201855
Epoch 5 completed out of 10 loss 51768.68316820264
Epoch 6 completed out of 10 loss 30962.474143002182
Epoch 7 completed out of 10 loss 26732.862425744534
Epoch 8 completed out of 10 loss 20446.793096524827
Epoch 9 completed out of 10 loss 23357.11720221102
Accuracy 0.9491


### sentiment analysis
positive or negative

https://pythonprogramming.net/static/downloads/machine-learning-data/pos.txt  
https://pythonprogramming.net/static/downloads/machine-learning-data/neg.txt

In [2]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import numpy as np
import random
import pickle
from collections import Counter

In [3]:
lemmatizer = WordNetLemmatizer()
hm_lines = 10000000

In [4]:
def create_lexicon(pos, neg):
    lexicon = []
    for fi in [pos, neg]:
        with open(fi, 'r') as f:
            countents = f.readlines()
            for l in countents[:hm_lines]:
                all_words = word_tokenize(l.lower())
                lexicon += list(all_words)
    
    lexicon = [lemmatizer.lemmatize(i) for i in lexicon]
    w_counts = Counter(lexicon)
    l2 = []
    for w in w_counts:
        if 1000 > w_counts[w] >50:
            l2.append(w)
    return l2

In [5]:
def sample_handling(sample, lexicon, classification):
    featureset = []
    with open(sample, 'r') as f:
        contents = f.readlines()
        for l in contents[:hm_lines]:
            current_words = word_tokenize(l.lower())
            current_words = [lemmatizer.lemmatize(i) for i in current_words]
            features = np.zeros(len(lexicon))
            for word in current_words:
                if word.lower() in lexicon:
                    index_value = lexicon.index(word.lower())
                    features[index_value] += 1
            features = list(features)
            featureset.append([features, classification])
    return featureset

In [6]:
def create_feature_sets_and_labels(pos, neg, test_size = 0.1):
    lexicon = create_lexicon(pos, neg)
    features = []
    features += sample_handling(pos, lexicon, [1,0])
    features += sample_handling(neg, lexicon, [0,1])
    random.shuffle(features)
    
    features = np.array(features)
    testing_size = int(test_size*len(features))
#     [[[0 1 1 0 0]],[0,1]]
#     [features, label]
    
    train_x = list(features[:,0][:-testing_size])
    train_y = list(features[:,1][:-testing_size])
    
    test_x = list(features[:,0][-testing_size:])
    test_y = list(features[:,1][-testing_size:])
    
    return train_x, train_y, test_x, test_y

In [7]:
train_x, train_y, test_x, test_y = create_feature_sets_and_labels('pos.txt', 'neg.txt')
with open('sentiment_set.pickle', 'wb') as f:
    pickle.dump([train_x, train_y, test_x, test_y], f)

In [8]:
#train_x, train_y, test_x, test_y = create_feature_sets_and_labels('pos.txt', 'neg.txt')

n_nodes_hl1 = 500
n_nodes_hl2 = 500
n_nodes_hl3 = 500

n_classes = 2
batch_size = 100

In [9]:
x = tf.placeholder('float', [None, len(train_x[0])])
y = tf.placeholder('float')

def neural_network_model(data):
    hidden_1_layer = {'weights':tf.Variable(tf.random_normal([len(train_x[0]), n_nodes_hl1])),
                      'biases':tf.Variable(tf.random_normal([n_nodes_hl1]))}

    hidden_2_layer = {'weights':tf.Variable(tf.random_normal([n_nodes_hl1, n_nodes_hl2])),
                      'biases':tf.Variable(tf.random_normal([n_nodes_hl2]))}

    hidden_3_layer = {'weights':tf.Variable(tf.random_normal([n_nodes_hl2, n_nodes_hl3])),
                      'biases':tf.Variable(tf.random_normal([n_nodes_hl3]))}

    output_layer = {'weights':tf.Variable(tf.random_normal([n_nodes_hl3, n_classes])),
                    'biases':tf.Variable(tf.random_normal([n_classes])),}


    l1 = tf.add(tf.matmul(data,hidden_1_layer['weights']), hidden_1_layer['biases'])
    l1 = tf.nn.relu(l1)

    l2 = tf.add(tf.matmul(l1,hidden_2_layer['weights']), hidden_2_layer['biases'])
    l2 = tf.nn.relu(l2)

    l3 = tf.add(tf.matmul(l2,hidden_3_layer['weights']), hidden_3_layer['biases'])
    l3 = tf.nn.relu(l3)

    output = tf.matmul(l3,output_layer['weights']) + output_layer['biases']

    return output

def train_neural_network(x):
    prediction = neural_network_model(x)
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = prediction, labels = y))
    optimizer = tf.train.AdamOptimizer().minimize(cost)
    
    hm_epochs = 10
    
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
# initialize_all_variables (from tensorflow.python.ops.variables) is deprecated and will be removed after 2017-03-02.
# Instructions for updating:
# Use `tf.global_variables_initializer` instead.
        for epoch in range(hm_epochs):
            epoch_loss = 0
            i = 0
            while i < len(train_x):
                start =i
                end = i+batch_size
                batch_x = np.array(train_x[start:end])
                batch_y = np.array(train_y[start:end])
                
                _, c = sess.run([optimizer, cost], feed_dict = {x:batch_x, y:batch_y})
                epoch_loss += c
                i += batch_size
            print('Epoch',epoch,'completed out of',hm_epochs,'loss',epoch_loss)
        
        correct = tf.equal(tf.argmax(prediction,1),tf.argmax(y,1))
        accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
        print('Accuracy', accuracy.eval({x:test_x, y:test_y}))
    

In [10]:
train_neural_network(x)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.

Epoch 0 completed out of 10 loss 210459.26403808594
Epoch 1 completed out of 10 loss 104545.35607910156
Epoch 2 completed out of 10 loss 63862.89129638672
Epoch 3 completed out of 10 loss 40182.00375366211
Epoch 4 completed out of 10 loss 26992.066230773926
Epoch 5 completed out of 10 loss 24898.681350708008
Epoch 6 completed out of 10 loss 20879.7331199646
Epoch 7 completed out of 10 loss 12511.25061416626
Epoch 8 completed out of 10 loss 10246.78747844696
Epoch 9 completed out of 10 loss 15598.579894065857
Accuracy 0.6210131


In [11]:
def create_feature_sets_and_labels_v2(pos, neg, test_size = 0.1):
    lexicon = create_lexicon(pos, neg)
    features = []
    features += sample_handling(pos, lexicon, 1)
    features += sample_handling(neg, lexicon, 0)
    random.shuffle(features)
    
    features = np.array(features)
    testing_size = int(test_size*len(features))
#     [[[0 1 1 0 0]],[0,1]]
#     [features, label]
    
    train_x = list(features[:,0][:-testing_size])
    train_y = list(features[:,1][:-testing_size])
    
    test_x = list(features[:,0][-testing_size:])
    test_y = list(features[:,1][-testing_size:])
    
    return train_x, train_y, test_x, test_y

In [12]:
train_x, train_y, test_x, test_y = create_feature_sets_and_labels_v2('pos.txt', 'neg.txt')

In [15]:
x_train = tf.keras.utils.normalize(train_x, axis = 1)
x_test = tf.keras.utils.normalize(test_x, axis = 1)
y_train = np.array(train_y)
y_test = np.array(test_y)
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(50, activation=tf.nn.relu)) #rectified linear
model.add(tf.keras.layers.Dense(50, activation=tf.nn.relu)) 
#model.add(tf.keras.layers.Dense(500, activation=tf.nn.relu)) 
model.add(tf.keras.layers.Dense(2, activation=tf.nn.softmax))

model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])
model.fit(x_train, y_train, epochs=10)

Epoch 1/10
9596/9596 [==============================] - 1s 77us/step - loss: 0.6512 - acc: 0.6111
Epoch 2/10
9596/9596 [==============================] - 0s 46us/step - loss: 0.5662 - acc: 0.6981
Epoch 3/10
9596/9596 [==============================] - 0s 47us/step - loss: 0.5382 - acc: 0.7234
Epoch 4/10
9596/9596 [==============================] - 0s 49us/step - loss: 0.5125 - acc: 0.7390
Epoch 5/10
9596/9596 [==============================] - 0s 50us/step - loss: 0.4766 - acc: 0.7687
Epoch 6/10
9596/9596 [==============================] - 0s 47us/step - loss: 0.4274 - acc: 0.8005
Epoch 7/10
9596/9596 [==============================] - 0s 50us/step - loss: 0.3697 - acc: 0.8393
Epoch 8/10
9596/9596 [==============================] - 0s 51us/step - loss: 0.3084 - acc: 0.8650
Epoch 9/10
9596/9596 [==============================] - 0s 48us/step - loss: 0.2505 - acc: 0.8939
Epoch 10/10
9596/9596 [==============================] - 0s 50us/step - loss: 0.2002 - acc: 0.9208


In [16]:
val_loss, val_acc = model.evaluate(x_test, y_test)
print(val_loss, val_acc)

1066/1066 [==============================] - 0s 93us/step
0.9617383171984224 0.660412757861905
